::: {.content-hidden}
# Toy Store KPI Report

### The Situation

You're a brand new Data Analyst for Maven Toys, a toy store chain with multiple store locations in Mexico.

### The Assignment

You have access to data containing transactional records from January 2022 - September 2023, along with information about products and store locations.

Your goal is to build a simple, interactive report that the leadership team can use to monitor key business metrics and high-level trends.

### The Objectives

1) Connect and profile the data
2) Create a relational model
3) Add calculated measures & fields
4) Build an interactive report

## Imports
:::

In [1]:
#| output: false
import polars as pl
import altair as alt

::: {.content-hidden}

## Objective 1: Connect and Profile the Data

1) Connect to the sales, products, stores, and calendar csv files
2) Review table columns, check for blank or null values, confirm that datatypes are accurately defined and identify any primary and foreign keys
3) Take a moment to profile the data. How many transactions were recorded? How many stores does Maven Toys operate? What are the lowest and highest priced products?
4) Add calculated columns in the calendar table for "start of month" and "start of week".

:::

In [2]:
#| output: false
sales = pl.scan_parquet("maven-toys-data/sales.parquet")
products = (
    pl.scan_csv("maven-toys-data/products.csv")
    .with_columns(
        pl.col("Product_Cost", "Product_Price")
        .str.replace("$", "", literal=True)
        .str.replace(" ", "", literal=True)
        .cast(pl.Float64)
    )
)
stores = pl.scan_csv("maven-toys-data/stores.csv", try_parse_dates=True)
calendar = (
    pl.scan_csv("maven-toys-data/calendar.csv")
    .with_columns(
        pl.col("Date").str.to_date(format="%m/%d/%Y")
    ).with_columns(
        Start_Month = pl.col("Date").dt.month_start(),
        Start_Week = (
            pl.col("Date")
            .dt.offset_by("1d")     # shift so Sunday becomes Monday
            .dt.truncate("1w")      # truncate (Monday-based)
            .dt.offset_by("-1d")    # shift back to Sunday
        )        
    )
)

::: {.content-hidden}

## Objective 2: Create a Relational Model

1) Load the tables to the data model and create relationships from the sales table to the product, stores, and calendar tables.
2) Confirm that you are following data modeling best practices. Your model should take the form of a star schema, with 1:Many relationships between fact and dimension tables.
3) Create a date hierarchy containing the "start of month", "start of week", and "date" fields.
4) Hide all foreign keys in the sales table from the report view.

:::

In [3]:
#| output: false
sales =  (
    sales
    .join(products, on="Product_ID", how="left")
    .join(stores, on="Store_ID", how="left")
    .join(calendar, on="Date", how="left")
    .with_columns(
       Date_Hierarchy = pl.struct(["Start_Month", "Start_Week", "Date"])
    ).drop("Store_ID", "Product_ID")
)

::: {.content-hidden}

## Objective 3: Add Calculated Measures & Fields

1) Create calculated columns in the sales table to pull in "cost" and "price" from the products table, then use those fields to calculate the revenue and profit for each transaction
2) Create measures to calculate the count of orders ("total orders"), sum of revenue ("total revenue") and sum of profit ("total profit")

:::

In [4]:
#| output: false
sales = (
    sales
    .with_columns(
        Revenue = pl.col("Product_Price") * pl.col("Units"),
        Profit = (pl.col("Product_Price") - pl.col("Product_Cost")) * pl.col("Units")
    )
)

In [5]:
#| output: false
total_orders = sales.select("Sale_ID").count().collect().item()
total_revenue = sales.select("Revenue").sum().collect().item()
total_profit = sales.select("Profit").sum().collect().item()

current_month_order = sales.filter(pl.col("Start_Month") == pl.col("Start_Month").max()).select("Sale_ID").count().collect().item()
current_month_revenue = sales.filter(pl.col("Start_Month") == pl.col("Start_Month").max()).select("Revenue").sum().collect().item()
current_month_profit = sales.filter(pl.col("Start_Month") == pl.col("Start_Month").max()).select("Profit").sum().collect().item()

::: {.content-hidden}

## Objective 4: Build and Interactive Report

1) Add KPI card visuals showing "total order", "total revenue", and ""total profit" for the current month, along with monthly trends for each metric.
2) Add a slicer to filter the report page by store location.
3) Add a bar chart showing "total orders" by product category, and a line chart showing "total revenue" with the date hierarchy on the x-axis.
4) Assemble the charts into a logical layout and adjust formatting, alignment, and polish to finalize the report as you see fit.

:::

In [6]:
#| output: false
def human_format(n, decimals=1):
    for unit in ["", "K", "M", "B", "T"]:
        if abs(n) < 1000:
            return f"{n:.{decimals}f}{unit}"
        n /= 1000

## Row {height=10%}

In [7]:
#| content: valuebox
#| title: "Total Orders"
dict(
    icon = "box-seam",
    color = "light",
    value = human_format(total_orders)
)

{'icon': 'box-seam', 'color': 'light', 'value': '829.3K'}

In [8]:
#| content: valuebox
#| title: "Total Revenue"
dict(
    icon = "currency-dollar",
    color = "light",
    value = human_format(total_revenue, 2)
)

{'icon': 'currency-dollar', 'color': 'light', 'value': '14.44M'}

In [9]:
#| content: valuebox
#| title: "Total Profit"
dict(
    icon = "currency-dollar",
    color = "success",
    value = human_format(total_profit, 2)
)

{'icon': 'currency-dollar', 'color': 'success', 'value': '4.01M'}

## Row {height=10%}

In [10]:
#| content: valuebox
#| title: "Current Month Orders"
dict(
    icon = "box-seam",
    color = "light",
    value = human_format(current_month_order)
)

{'icon': 'box-seam', 'color': 'light', 'value': '41.8K'}

In [11]:
#| content: valuebox
#| title: "Current Month Revenue"
dict(
    icon = "currency-dollar",
    color = "light",
    value = human_format(current_month_revenue, 2)
)

{'icon': 'currency-dollar', 'color': 'light', 'value': '658.19K'}

In [12]:
#| content: valuebox
#| title: "Current Month Profit"
dict(
    icon = "currency-dollar",
    color = "success",
    value = human_format(current_month_profit, 2)
)

{'icon': 'currency-dollar', 'color': 'success', 'value': '180.44K'}

## Row {height=20%}

In [13]:
#| output: false
monthly_summary = (
    sales
    .group_by("Start_Month")
    .agg(
        Monthly_Orders = pl.len(),
        Monthly_Revenue = pl.col("Revenue").sum(),
        Monthly_Profit = pl.col("Profit").sum()
    )
)

In [14]:
#| title: Monthly Orders
alt.Chart(monthly_summary.collect()).mark_area().encode(
    x=alt.X("Start_Month:T", title="Month"),
    y=alt.Y("Monthly_Orders:Q", scale=alt.Scale(zero=False), title="Orders")
)

alt.Chart(...)

In [15]:
#| title: Monthly Revenue
alt.Chart(monthly_summary.collect()).mark_area().encode(
    x=alt.X("Start_Month:T", title="Month"),
    y=alt.Y("Monthly_Revenue:Q", scale=alt.Scale(zero=False), title="Revenue")
)

alt.Chart(...)

In [16]:
#| title: Monthly Profit
alt.Chart(monthly_summary.collect()).mark_area().encode(
    x=alt.X("Start_Month:T", title="Month"),
    y=alt.Y("Monthly_Profit:Q", scale=alt.Scale(zero=False), title="Profit")
)

alt.Chart(...)